In [3]:
# this is an analysis of online purchases
# Data are retrieved from theextremely well-made kaggle kernel available at https://www.kaggle.com/fabiendaniel/customer-segmentation, where I also learned new visualization and techiques and python tricks

import pandas as pd

df = pd.read_csv('data.csv', encoding = "ISO-8859-1",
                 dtype={'CustomerID': str,'InvoiceID': str})
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

display(df.head())
display(df.describe())


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


,Quantity,UnitPrice
count,541909.000000,541909.000000
mean,9.552250,4.611114
std,218.081158,96.759853
min,-80995.000000,-11062.060000
25%,1.000000,1.250000
50%,3.000000,2.080000
75%,10.000000,4.130000
max,80995.000000,38970.000000


In [4]:
# let's look at missing values
tab_info = pd.DataFrame(df.dtypes).T.rename(index={0:'column type'}) # make new df with data types and missing val
tab_info = tab_info.append(pd.DataFrame(df.isnull().sum()).T.rename(index={0:'null values (nb)'}))
tab_info = tab_info.append(pd.DataFrame(df.isnull().sum()/df.shape[0]*100).T.rename(index={0:'null values (%)'}))
display(tab_info)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
column type,object,object,object,int64,datetime64[ns],float64,object,object
null values (nb),0,0,1454,0,0,0,135080,0
null values (%),0,0,0.268311,0,0,0,24.9267,0


In [5]:
# for now, let's just remove incomplete data entries. later we might look more into it and consider including them
df.dropna(axis = 0, subset = ['CustomerID', ], inplace = True)
print('Dataframe dimensions:', df.shape) # note we are dropping >130k instances... 

Dataframe dimensions: (406829, 8)


In [54]:
# let's now visualize where the orders come from
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from plotly.offline import iplot
import plotly.plotly as py

temp = df[['InvoiceNo', 'Country']].groupby([ 'InvoiceNo', 'Country']).count()

temp = temp.reset_index(drop = False)
#display(temp)

countries = temp['Country'].value_counts()
display((countries.index))

print('Countries in dataframe:', len(countries))



data = dict(type='choropleth',
locations = countries.index,
locationmode = 'country names', z = countries,
text = countries.index, colorbar = {'title':'Order nb.'},
colorscale=[[0, 'rgb(224,255,255)'],
            [0.01, 'rgb(166,206,227)'], [0.02, 'rgb(31,120,180)'],
            [0.03, 'rgb(178,223,138)'], [0.05, 'rgb(51,160,44)'],
            [0.10, 'rgb(251,154,153)'], [0.20, 'rgb(255,255,0)'],
            [1, 'rgb(227,26,28)']],    
reversescale = False)

layout = dict(title='Number of orders per country',
geo = dict(showframe = True, projection={'type':'Mercator'}))

choromap = go.Figure(data = [data], layout = layout)
iplot(choromap, validate=False)




Index(['United Kingdom', 'Germany', 'France', 'EIRE', 'Belgium', 'Spain',
       'Netherlands', 'Switzerland', 'Portugal', 'Australia', 'Italy',
       'Finland', 'Sweden', 'Norway', 'Channel Islands', 'Japan', 'Poland',
       'Denmark', 'Cyprus', 'Austria', 'Malta', 'Singapore', 'Unspecified',
       'USA', 'Iceland', 'Israel', 'Canada', 'Greece', 'European Community',
       'Czech Republic', 'Lithuania', 'United Arab Emirates', 'Bahrain',
       'Saudi Arabia', 'Lebanon', 'RSA', 'Brazil'],
      dtype='object')

Countries in dataframe: 37


In [48]:
"""

              autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '$',
            title = 'GDP<br>Billions US$'),



data = dict(type='choropleth',
locations = countries.index,
locationmode = 'country names', z = countries,
text = countries.index, colorbar = {'title':'Order nb.'},
colorscale=[[0, 'rgb(224,255,255)'],
            [0.01, 'rgb(166,206,227)'], [0.02, 'rgb(31,120,180)'],
            [0.03, 'rgb(178,223,138)'], [0.05, 'rgb(51,160,44)'],
            [0.10, 'rgb(251,154,153)'], [0.20, 'rgb(255,255,0)'],
            [1, 'rgb(227,26,28)']],    
reversescale = False)

layout = dict(title='Number of orders per country',
geo = dict(showframe = True, projection={'type':'Mercator'}))


choromap = go.Figure(data = [data], layout = layout)
iplot(choromap, validate=False)

fig = dict( data=[data], layout=layout )
py.iplot( fig, validate=False, filename='d3-world-map' )
"""

"\n\n              autocolorscale = False,\n        reversescale = True,\n        marker = dict(\n            line = dict (\n                color = 'rgb(180,180,180)',\n                width = 0.5\n            ) ),\n        colorbar = dict(\n            autotick = False,\n            tickprefix = '$',\n            title = 'GDP<br>Billions US$'),\n\n\n\ndata = dict(type='choropleth',\nlocations = countries.index,\nlocationmode = 'country names', z = countries,\ntext = countries.index, colorbar = {'title':'Order nb.'},\ncolorscale=[[0, 'rgb(224,255,255)'],\n            [0.01, 'rgb(166,206,227)'], [0.02, 'rgb(31,120,180)'],\n            [0.03, 'rgb(178,223,138)'], [0.05, 'rgb(51,160,44)'],\n            [0.10, 'rgb(251,154,153)'], [0.20, 'rgb(255,255,0)'],\n            [1, 'rgb(227,26,28)']],    \nreversescale = False)\n\nlayout = dict(title='Number of orders per country',\ngeo = dict(showframe = True, projection={'type':'Mercator'}))\n\n\nchoromap = go.Figure(data = [data], layout = layo